In [1]:
from testutil import change_to_root

change_to_root()

Current working directory: /Users/David/Desktop/NORS_Outages_Completed/tests
Changed to project root directory: /Users/David/Desktop/NORS_Outages_Completed


In [2]:
from data_prep import DataPreparer
dataprep = DataPreparer(start_year=2016, end_year=2023, folder='datasets', outage_file_name='outage_data.csv', financial_file_name='ppe.csv')
dataprep.save_to_csv(folder='datasets')

Initialized with year range 2016 to 2023.
Linking Data...
Data aggregated and counted by company, year, and quarter successfully.
Reading finance data from: datasets/ppe.csv
The data file is missing the 'Company' column; attempting to clean data...
Quarter and Year columns created successfully.
Finance data filtered successfully.
Finance data unpivoted successfully.
Data linked successfully.


In [3]:
from data_prep import FinancialDataTransformer

f = FinancialDataTransformer(financial_data_file_name='ppe.csv', folder='datasets')
f.get_financial_data()

Reading finance data from: datasets/ppe.csv
The data file is missing the 'Company' column; attempting to clean data...
Quarter and Year columns created successfully.
Finance data filtered successfully.
Finance data unpivoted successfully.


,Company,Quarter,PP&E,Year
0,AT&T Inc.,Q1,47.132000,2000
1,"Frontier Communications Parent, Inc.",Q1,2.946976,2000
2,"Lumen Technologies, Inc.",Q1,2.232390,2000
3,Comcast Corporation,Q1,4.764600,2000
4,Verizon Communications Inc.,Q1,39.749000,2000
...,...,...,...,...
667,"Lumen Technologies, Inc.",Q4,19.606000,2023
668,Comcast Corporation,Q4,58.165000,2023
669,Verizon Communications Inc.,Q4,133.036000,2023
670,"T-Mobile US, Inc.",Q4,72.038000,2023


In [4]:
from data_prep import OutageDataProcessor

o = OutageDataProcessor(outage_file_name='report_edited.csv', start_year=2021, end_year=2023,folder='datasets')
o.get_outage_frequency()

Initialized with year range 2021 to 2023.
Data aggregated and counted by company, year, and quarter successfully.


,Company,Year,Quarter,Count
0,AT&T Inc.,1999,Q4,2
1,AT&T Inc.,2001,Q1,2
2,AT&T Inc.,2003,Q1,2
3,AT&T Inc.,2004,Q1,2
4,AT&T Inc.,2004,Q2,2
...,...,...,...,...
791,Verizon Communications Inc.,2022,Q4,7390
792,Verizon Communications Inc.,2023,Q1,7772
793,Verizon Communications Inc.,2023,Q2,9052
794,Verizon Communications Inc.,2023,Q3,9919


In [24]:
from data_prep import DataPreparer
d = DataPreparer(outage_file_name='report_edited.csv',financial_file_name='ppe.xlsx', start_year=2021, end_year=2023,folder='datasets')



Initialized with year range 2021 to 2023.
Linking Data...
Data aggregated and counted by company, year, and quarter successfully.
Reading finance data from: datasets/ppe.xlsx
The data file is missing the 'Company' column; attempting to clean data...
Quarter and Year columns created successfully.
Finance data filtered successfully.
Finance data unpivoted successfully.
Data linked successfully.


In [ ]:
d.save_to_csv(folder='datasets')

In [63]:
df = d.get_data()
df2 = df[df['Year'] > 2020]
df2


,Company,Year,Quarter,Count,PP&E
68,AT&T Inc.,2021,Q1,7450,149.022
69,AT&T Inc.,2021,Q2,8183,147.981
70,AT&T Inc.,2021,Q3,9195,148.760
71,AT&T Inc.,2021,Q4,8878,143.473
72,AT&T Inc.,2022,Q1,8195,151.100
...,...,...,...,...,...
496,Verizon Communications Inc.,2022,Q4,7390,133.564
497,Verizon Communications Inc.,2023,Q1,7772,132.934
498,Verizon Communications Inc.,2023,Q2,9052,132.615
499,Verizon Communications Inc.,2023,Q3,9919,132.576


In [64]:
df2['Outage per PP&E'] = df['Count'] / df['PP&E']


/var/folders/5b/zr381m555nndpqxs7__k9sv40000gq/T/ipykernel_24207/2143248676.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [74]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import plotly.graph_objects as go

# Ensure your DataFrame, df2, is set up correctly. Example setup:
# df2 = pd.DataFrame({
#     'Count': [values],
#     'PP&E': [values],
#     'Company': ['Company A', 'Company B', ...]  # Sample company names
# })

# Set up quantile regression
formula = 'Count ~ Q("PP&E")'
quantiles = [0.05, 0.5,  0.95]
results = {}
x = pd.DataFrame({'PP&E': np.linspace(df2['PP&E'].min(), df2['PP&E'].max(), 100)})

# Plotly Figure
fig = go.Figure()

# Plotting each quantile regression
for qt in quantiles:
    mod = smf.quantreg(formula, df2)
    res = mod.fit(q=qt)
    y_pred = res.predict(x)
    fig.add_trace(go.Scatter(x=x['PP&E'], y=y_pred, mode='lines', name=f'{qt} Quantile'))

# Adding original data points, colored by company
for company in df2['Company'].unique():
    company_data = df2[df2['Company'] == company]
    fig.add_trace(go.Scatter(x=company_data['PP&E'], y=company_data['Count'], mode='markers', name=company,
                             marker=dict(size=10, line=dict(width=2), opacity=0.8)))

# Updating layout
fig.update_layout(
    title='Quantile Regression Lines for Count vs. PP&E by Company',
    xaxis_title='PP&E',
    yaxis_title='Count',
    legend_title="Quantiles and Companies",
    plot_bgcolor='white'
)

# Show the figure
fig.show()


In [67]:
# Setting up the plotting area
plt.figure(figsize=(10, 6))
sns.lineplot(x='Quantile', y='PP&E', data=coef_df, marker='o')

# Adding titles and labels
plt.title('Effect of PP&E on Count Across Different Quantiles')
plt.xlabel('Quantiles')
plt.ylabel('Coefficient of PP&E')

# Display the plot
plt.grid(True)
plt.show()


NameError: name 'coef_df' is not defined

<Figure size 1000x600 with 0 Axes>

In [8]:
df.to_csv('datasets/prepared_data.csv', index=False)

In [9]:
from visualization import PlotlyVisualizer

vis = PlotlyVisualizer('prepared_data.csv', 'datasets')
vis.plot_average_outage_vs_ppe(enable_ci=True)

In [10]:
vis.plot_granular_outage_vs_ppe(enable_pi=True)

In [11]:
vis.plot_outage_per_ppe_over_time()

In [12]:
vis.plot_outage_per_ppe_boxplot(include_grand_total=True)